<a href="https://colab.research.google.com/github/hokou/data-course-sample/blob/wk01/wk01_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [3]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [4]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-26 13:02:41--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv’

All_Beauty.csv      100%[===================>]  14.78M  18.5MB/s    in 0.8s    

2021-12-26 13:02:42 (18.5 MB/s) - ‘All_Beauty.csv’ saved [15499476/15499476]

--2021-12-26 13:02:42--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz’

meta_All_Beauty.jso 100%[===================>]   9.85M  14.1MB/s    in 0.7s    

202

In [5]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [6]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [7]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [8]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

In [9]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [10]:
ratings_trainings_3m = ratings[
    (ratings['DATE'] >= '2018-06-01') & 
    (ratings['DATE'] < '2018-09-01')
]

In [11]:
print("train:", len(ratings_trainings))
print("test:", len(ratings_testings))
print("train_3M:", len(ratings_trainings_3m))

train: 370752
test: 590
train_3M: 7027


## 產生推薦

In [37]:
def recommender_by_asin(training_data, k=10):
  '''
  排序數量最多的 K 個商品
  '''
  data = training_data.groupby(by=['asin']).size().sort_values(ascending=False)
  data_list = data.index.tolist()
  data_list = data_list[:k]
  return data_list

recommender_list = recommender_by_asin(ratings_trainings, k=10)
print(recommender_list)

['B000FOI48G', 'B000GLRREU', '1620213982', 'B001QY8QXM', 'B01DKQAXC0', 'B00W259T7G', 'B006IB5T4W', 'B00005JS5C', 'B0012Y0ZG2', 'B000WYJTZG']


In [53]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    Your Code
    '''
    recommender_list = recommender_by_asin(training_data, k)
    recommendations = {x: recommender_list for x in users}

    return recommendations

In [55]:
ratings_by_user_1 = recommender(ratings_trainings, users, k=10)
ratings_by_user_2 = recommender(ratings_trainings, users, k=50)
ratings_by_user_3 = recommender(ratings_trainings_3m, users, k=10)
ratings_by_user_4 = recommender(ratings_trainings_3m, users, k=50)
print(ratings_by_user_1)
print(ratings_by_user_2)
print(ratings_by_user_3)
print(ratings_by_user_4)

{'A100XQFWKQ30O2': ['B000FOI48G', 'B000GLRREU', '1620213982', 'B001QY8QXM', 'B01DKQAXC0', 'B00W259T7G', 'B006IB5T4W', 'B00005JS5C', 'B0012Y0ZG2', 'B000WYJTZG'], 'A103T1QOGFCSEH': ['B000FOI48G', 'B000GLRREU', '1620213982', 'B001QY8QXM', 'B01DKQAXC0', 'B00W259T7G', 'B006IB5T4W', 'B00005JS5C', 'B0012Y0ZG2', 'B000WYJTZG'], 'A106UKKSJ2KXPF': ['B000FOI48G', 'B000GLRREU', '1620213982', 'B001QY8QXM', 'B01DKQAXC0', 'B00W259T7G', 'B006IB5T4W', 'B00005JS5C', 'B0012Y0ZG2', 'B000WYJTZG'], 'A10A7GV4D5A11V': ['B000FOI48G', 'B000GLRREU', '1620213982', 'B001QY8QXM', 'B01DKQAXC0', 'B00W259T7G', 'B006IB5T4W', 'B00005JS5C', 'B0012Y0ZG2', 'B000WYJTZG'], 'A1119JJ37ZLB8R': ['B000FOI48G', 'B000GLRREU', '1620213982', 'B001QY8QXM', 'B01DKQAXC0', 'B00W259T7G', 'B006IB5T4W', 'B00005JS5C', 'B0012Y0ZG2', 'B000WYJTZG'], 'A113UOOLBSZN52': ['B000FOI48G', 'B000GLRREU', '1620213982', 'B001QY8QXM', 'B01DKQAXC0', 'B00W259T7G', 'B006IB5T4W', 'B00005JS5C', 'B0012Y0ZG2', 'B000WYJTZG'], 'A12M4U7WK4ALCR': ['B000FOI48G', 'B000G

## 結果評估

In [56]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

In [59]:
score_1 = evaluate(ratings_testings_by_user, ratings_by_user_1)
score_2 = evaluate(ratings_testings_by_user, ratings_by_user_2)
score_3 = evaluate(ratings_testings_by_user, ratings_by_user_3)
score_4 = evaluate(ratings_testings_by_user, ratings_by_user_4)
print("2000-01-10 - 2018-08-31:TOP10",score_1)
print("2000-01-10 - 2018-08-31:TOP50",score_2)
print("2018-06-01 - 2018-08-31:TOP10",score_3)
print("2018-06-01 - 2018-08-31:TOP50",score_4)

2000-01-10 - 2018-08-31:TOP10 0.08305084745762711
2000-01-10 - 2018-08-31:TOP50 0.10508474576271186
2018-06-01 - 2018-08-31:TOP10 0.13389830508474576
2018-06-01 - 2018-08-31:TOP50 0.30338983050847457
